In [1]:
import tfim
import numpy as np
from itertools import groupby
import symmetry_types

In [2]:
PBC = True
L = [9]

lattice = tfim.Lattice(L, PBC)
N = lattice.N
J = 1

seed = 7
Jij = tfim.Jij_instance(N,J,dist="bimodal",seed=seed,even=True)

basis = tfim.IsingBasis(lattice)

Energies = -1 * (tfim.JZZ_SK_ME(basis,Jij))

100% |########################################################################|


In [3]:
def grouped_configs(basis, Jij):
    '''function returns arrays of integers that have the same energy when converted to 
    spin configurations. excludes integers that are spin inversions of each other'''
    Energies = -1 * (tfim.JZZ_SK_ME(basis,Jij))
    sorted_indices = np.argsort(Energies)
    sorted_energies = Energies[sorted_indices]
    
    split_list = []                                #Breaks up energy list by value
    for key, group in groupby(sorted_energies):
        split_list.append(list(group))
                                            
    num_list = []
    for x in range(0, len(split_list)):           
        index = 0
        for y in range(0, x+1):
            index = index + len(split_list[y]) 
        start = (index - len(split_list[x]))
        entry = sorted_indices[start:index]
        entry.sort()
        #entry = entry[:len(entry)/2]
        num_list.append(entry)
        #num_list is a list of lists that contain indices with the same energy, excluding spin inversions
    return num_list

In [4]:
grouped_configs = grouped_configs(basis,Jij)

100% |########################################################################|


[array([147, 154, 155, 179, 183, 210, 211, 215, 218, 247, 264, 293, 296,
       300, 301, 328, 332, 356, 357, 364]), array([  8,  12,  18,  19,  23,  24,  26,  36,  37,  44,  53,  55,  68,
        70,  76,  82,  86,  87, 100, 146, 151, 152, 181, 187, 202, 214,
       219, 231, 243, 268, 280, 292, 297, 309, 324, 330, 359, 360, 365,
       411, 424, 425, 429, 435, 441, 443, 456, 458, 467, 474, 475, 485,
       487, 488, 492, 493, 499, 503]), array([  4,  10,  16,  20,  21,  22,  27,  28,  39,  40,  45,  51,  52,
        66,  71,  72,  74,  78,  83,  84,  90, 101, 102, 103, 108, 119,
       136, 138, 153, 159, 165, 167, 168, 172, 173, 177, 178, 184, 185,
       186, 189, 191, 194, 198, 199, 200, 204, 206, 216, 222, 223, 228,
       229, 230, 236, 242, 245, 246, 265, 266, 269, 275, 281, 282, 283,
       288, 289, 295, 305, 307, 311, 312, 313, 317, 320, 322, 325, 326,
       327, 333, 334, 338, 339, 343, 344, 346, 352, 358, 373, 375, 392,
       403, 408, 409, 410, 421, 427, 428, 433, 437, 

In [5]:
def list_transformations(grouped_configs, max_energy, N):
    """function takes in grouped_configs, which is a list of configuration integers grouped by energy level, and
    returns a list of all transformations that occur in each energy level 
    
    when only finding transformations for a few energy levels, the ground state corresponds to max_energy=1, using
    'all' will go through all energy levels"""
    
    transformation_list = []
    if max_energy == 0:
        max_energy = len(grouped_configs)
    for entry in grouped_configs[:max_energy]:
        single_list = []
         
        if len(entry)>1: 
            index = 0
            for num in entry[:-1]:
                index = index + 1
                num = basis.state(num)
                
                for num2 in entry[index:]:
                    
                    num2 = basis.state(num2)
                    diff = num - num2
                    
                    nonzero = np.nonzero(diff)[0]
                    if nonzero.size > N/2:         #This is the piece that will throw out hamming distances greater than N/2
                        continue
                        
                    else:
                        substate1 = basis.index(num[nonzero])
                        substate2 = basis.index(num2[nonzero])

                        transformation = [nonzero, substate1, substate2]
                        single_list.append(transformation)

                        
                    #single_list.append(transformation)
        transformation_list.append(single_list)       
        #a list with nonzero array, then the integers from the binary number formed by applying the nonzero array
    return transformation_list


In [6]:
transformations = (list_transformations(grouped_configs, 0, L[0]))

In [8]:
def symmetry(transformation_list, grouped_configs, basis):
    """function takes in a list of all transformations grouped by energy level and grouped_configs and returns a
    list of symmetries without duplicates"""
    
    symm_list = []
    level_index = 0
    for energy_level in transformation_list:
        if len(energy_level) == 0:
            continue
        level_index = level_index + 1
        transformation_index = -1
        for transformation in energy_level:
            transformation_index = transformation_index + 1
            switched_index = transformation [0]
            int1 = transformation[1]
            int2 = transformation[2] #These are integers from the binary number taken out by switched_index
            
            is_a_symmetry = True
            
            for energy_level2 in grouped_configs[:]: #used to start at level_index I think
                if  not is_a_symmetry: 
                    break
                
                for configuration in energy_level2:   #configuration is the integer
                    binary = basis.state(configuration) #convert to binary
                    switched = basis.index(binary[switched_index])  #convert flipped spins to integers like config1/2
                    if switched == int1 or switched == int2:
                        #now flip the spins in those spots and see if the new configuration is contained in
                        for value in switched_index:
                            binary[value] = (binary[value]-1)*-1
                        binary = basis.index(binary)  #now, since we have done the transformation, if this value is not in the energy, we can throw out the symmetry
                        if binary in energy_level2:
                            break
                        else:
                            is_a_symmetry = False
                            break 
                    else:
                        continue
            if is_a_symmetry:
                repeat = False
                for trans in symm_list:
                        if np.array_equal(transformation[0], trans[0]) and transformation[1]==trans[1]:
                            repeat = True
                            break
                if repeat == False:
                    symm_list.append(transformation)
                        
    return symm_list

In [9]:
symmetries = symmetry(transformations, grouped_configs)

[[array([3, 5, 6, 8]), 4, 11], [array([0, 1, 3, 8]), 0, 15], [array([0, 1, 5, 6]), 1, 14], [array([1, 2, 4, 7]), 2, 13], [array([0, 2, 4, 7]), 2, 13], [array([1, 2, 4, 7]), 1, 14], [array([0, 2, 4, 7]), 1, 14], [array([3, 5, 6, 8]), 7, 8], [array([0, 1, 5, 6]), 7, 8], [array([0, 1, 3, 8]), 6, 9]]


In [10]:
def list_sorter(transformations, N):
    '''Takes a list of transformations and sorts them by hamming distance
    Returns a list of transformations sorted by hamming distance without any duplicates'''
    sorted_list = []
    for length in range(2, N):
        if length%2 != 0:
            one_length = []
            for transformation in transformations:
                array = transformation[0]
                if array.size == length:
                        one_length.append(transformation)
                else:
                    continue
            sorted_list.append(one_length)
        else:
            one_length = []
            for transformation in transformations:
                array = transformation[0]
                if array.size == length:
                        one_length.append(transformation)
                else:
                    continue
            sorted_list.append(one_length)
    return sorted_list

In [11]:
sorted_list = list_sorter(symmetries, L[0])


[[], [], [[array([3, 5, 6, 8]), 4, 11], [array([0, 1, 3, 8]), 0, 15], [array([0, 1, 5, 6]), 1, 14], [array([1, 2, 4, 7]), 2, 13], [array([0, 2, 4, 7]), 2, 13], [array([1, 2, 4, 7]), 1, 14], [array([0, 2, 4, 7]), 1, 14], [array([3, 5, 6, 8]), 7, 8], [array([0, 1, 5, 6]), 7, 8], [array([0, 1, 3, 8]), 6, 9]], [], [], [], []]


In [12]:
def remove_combo(sorted_list):
    '''Function removes transformations of Hamming distance 4 that are combinations of hamming distance 2 transformations
    Eventually, I might modify it to remove combinations that are longer though that might not become a big enough issue
    Input must be a list with Hamming distance 2 at index 0 and distance 4 at index 1'''
    
    length_2 = sorted_list[0]
    length_4 = sorted_list[1]
    transform_index = 0
    for transformation in length_2:
        transform_index += 1
        array = transformation[0]
        for transformation2 in length_2[transform_index:]:
            array2 = transformation2[0]
            if (array == array2).any() or array[0]==array2[1] or array[1]==array2[0]:
                continue
            else:
                #print 't: ',transformation, ' t2: ', transformation2
                combo = np.concatenate((array, array2))
                sorted_indices = np.argsort(combo)
                sorted_combo = combo[sorted_indices]
                #print 'sort combo: ', sorted_combo
    
                for arr in transformation[1:]:
            
                    arr = basis.state(arr)[-2:]
                    arr2 = basis.state(transformation2[1])[-2:]
                    binary = np.concatenate((arr, arr2))
                    sorted_indices = np.argsort(combo)
                    binary = binary[sorted_indices]
                    #print 'sorted binary: ', binary
                    int1 = basis.index(binary)
                    #print 'int1: ', int1
                    
                    t_combo_index = -1
                    for t_combo in sorted_list[1]:
                        t_combo_index += 1
                        #print 't_combo: ',t_combo
                        if (t_combo[0] == sorted_combo).all() and (int1==t_combo[1] or int1==t_combo[2]):
                            del sorted_list[1][t_combo_index]
                            break
                        else:
                            continue
    return sorted_list          

In [13]:
no_combo = remove_combo(sorted_list)

[[], [], [[array([3, 5, 6, 8]), 4, 11], [array([0, 1, 3, 8]), 0, 15], [array([0, 1, 5, 6]), 1, 14], [array([1, 2, 4, 7]), 2, 13], [array([0, 2, 4, 7]), 2, 13], [array([1, 2, 4, 7]), 1, 14], [array([0, 2, 4, 7]), 1, 14], [array([3, 5, 6, 8]), 7, 8], [array([0, 1, 5, 6]), 7, 8], [array([0, 1, 3, 8]), 6, 9]], [], [], [], []]


In [15]:
def symmetry_sorter(symmetries, sortedsym):
    '''Sorts the symmetries I have found into swap, anti-swap, and other'''
    swap_symmetries = []
    anti_swap = []
    other = []
    
    swap_sym = sortedsym[0]
    anti_swap_sym = sortedsym[1]
    
    sorted_swap = []
    sorted_antiswap = []
 
    for tup in swap_sym:
        tup = sorted(tup)
        sorted_swap.append(tuple(tup))
        
    
    for tup2 in anti_swap_sym:
        tup2 = sorted(tup2)
        sorted_antiswap.append(tuple(tup2))
        
    for length in symmetries:
        for transformation in length:
            array = tuple(transformation[0])
            if array in sorted_swap:
                swap_symmetries.append(transformation)
            elif array in sorted_antiswap:
                anti_swap.append(transformation)
            else:
                other.append(transformation)
    if len(swap_symmetries) != 0:
        print 'swap_symmetries: ', swap_symmetries
    if len(swap_symmetries) != 0:
        print 'anti_swap: ', anti_swap
    if len(other) != 0:
        print 'other: ', other
    return swap_symmetries, anti_swap, other

In [14]:
symmetries2 = symmetry_types.main(seed, L[0])

([(0, 1, 3, 4, 6, 7), (0, 1, 4, 3, 6, 7)], [(0, 8, 1, 3), (1, 3, 7, 5, 6, 8), (1, 5, 7, 3, 6, 8)])


In [16]:
symmetry_types = symmetry_sorter(no_combo, symmetries2)

swap_symmetries:  []
anti_swap:  [[array([0, 1, 3, 8]), 0, 15], [array([0, 1, 3, 8]), 6, 9]]
other:  [[array([3, 5, 6, 8]), 4, 11], [array([0, 1, 5, 6]), 1, 14], [array([1, 2, 4, 7]), 2, 13], [array([0, 2, 4, 7]), 2, 13], [array([1, 2, 4, 7]), 1, 14], [array([0, 2, 4, 7]), 1, 14], [array([3, 5, 6, 8]), 7, 8], [array([0, 1, 5, 6]), 7, 8]]
